In [31]:
import re

def parse_sc_fermi_output(output):
    data = {}
    regex = re.compile('SC Fermi level :\s+([-\d\.]+)')
    data['e_fermi'] = float(regex.findall(output)[0])
    regex = re.compile('(\w[()\w\s]+[\w\)])\s+:\s+([-+\d.E]+)\s+cm\^-3')
    data['defect species concentrations'] = {}
    for name, conc in regex.findall(output):
        data['defect species concentrations'][name] = float(conc)
    divider = '-'*56 +'\n'
    data['defect species'] = {}
    for defect_data in output.split(divider)[1:]:
        split_defect_data = defect_data.split(':')
        defect_species = split_defect_data[0].strip()
        data[defect_species] = {}
        for d in split_defect_data[2:]:
            values = [ float(s) for s in d.split() ]
            charge = values[0]
            data[defect_species][charge] = {}
            data[defect_species][charge]['concentration'] = values[1]
            data[defect_species][charge]['percentage'] = values[2]
    return data
    

In [33]:
import os
sc_fermi_exec = '/Users/bjm42/source/sc-fermi/upload-sourceforge/sc-fermi'
output = os.popen(sc_fermi_exec).read()
print(output)
sc_fermi_output = parse_sc_fermi_output(output)

**************************************************************

   SSSS    CCCC      FFFFFF  EEEEEE   RRRR   MM     MM  IIIII
  SS   S  CC   C     FF      EE      RR   R  MMM   MMM    I
  SS      CC         FF      EE      RR  R   M MM MM M    I
   SSSS   CC     --- FFFFFF  EEEEEE  RRRR    M  MMM  M    I
      SS  CC         FF      EE      R   R   M   M   M    I
  S   SS  CC   C     FF      EE      R   RR  M   M   M    I
   SSSS    CCCC      FF      EEEEEE  R   RR  M   M   M  IIIII

Energies in eV, temperature in Kelvin, DOS in states/unit cell

------
j.buckeridge@ucl.ac.uk 2018
**************************************************************

unitcell.dat found...
(Should be cell for which DOS was determined!)
 
Volume of cell:      57.788884 A^3
 
Found non-spin polarised system...
 
Number of electrons in system:      18.000000
         Energy gap of system:       0.808400 eV
                 Temperature :     100.000000 K
     Number of defect species:     2
 
Defects found:
Name  

In [34]:
# py-sc-fermi

from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.inputs import inputs_from_files
from py_sc_fermi.dos import DOS

import numpy as np

In [35]:
example_dir = './'

unitcell_filename = f'{example_dir}/unitcell.dat'
totdos_filename = f'{example_dir}/totdos.dat'
input_fermi_filename = f'{example_dir}/input-fermi.dat'

inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                           input_fermi_filename=input_fermi_filename, 
                           totdos_filename=totdos_filename)

Volume of cell: 57.78888409009906 A^3
Found non-spin polarised system...
Number of electrons in system: 18
Energy gap of system: 0.8084 eV
Temperature: 100.0 K
Number of defect species: 2
Reading non-spin-polarised DOS


In [36]:
defect_system = DefectSystem( defect_species=inputs['defect_species'],
                              volume=inputs['volume'],
                              dos=inputs['dos'], # DOS object
                              temperature=inputs['temperature'] )

print(defect_system)

defect_system.report()

DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 100.0 K

Contains defect species:

V_Ga, nsites=1
  q=+0.0, e=2.4451, deg=1
  q=-1.0, e=0.0265, deg=1
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

SC Fermi level :      -0.027501064253249576  (eV)

Concentrations:
n (electrons)  : 2.585422888323457e-24 cm^-3
p (holes)      : 3.2856773709964726e+19 cm^-3
V_Ga           : 3.2856773282043056e+19 cm^-3
Ga_Sb          : 2.159971485632336e-85 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
V_Ga       : Charge Concentration(cm^-3) Total
           :  0.0  1.024893e-101          0.00 
           : -1.0  3.285677e+19          100.00 
           : -2.0  1.540824e-99          0.00 
           : -3.0  1.864311e-119          0.00 
--------------------------------------------------

/Users/bjm42/.pyenv/versions/3.7.0/lib/python3.7/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/dos.py:86: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/dos.py:89: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)


In [6]:
defect_system.defect_species_by_name('V_Ga').tl_profile(ef_min=defect_system.dos.emin(), ef_max=defect_system.dos.emax())

array([[-20.65969874,   2.4451    ],
       [ -2.4186    ,   2.4451    ],
       [  1.34405   ,  -1.31755   ],
       [ 11.38265078, -31.43335235]])